In [1]:
import numpy as np

path_prefix = "./all/data/"
word_to_index = {} # map word to index
g_word_list = [] # sorted by frequence low to high
category_word_dict = {}
item_word_dict = {}
skip_words = set(['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'as', 'an', 'and', 'are', 'at', 'by', 'down', 'for', 'from', 'front', 'has', 'in', 'is', 'it', 'near', 'next', 'of', 'on', 'out', 'over', 'several', 'sits', 'some', 'that', 'the', 'there', 'through', 'to', 'top', 'up', 'while', 'with'])
train_des = [] #0/1/2 array of word
test_des = [] #0/1/2 array of word
train_feature_list = [] # list of feature_array by index
test_feature_list = []

In [2]:
def read_file_to_string(file_name):
    f = open(file_name, "r")
    content = f.read()
    f.close()
    content = content.lower()
    content = content.replace(",", " ").replace(".", " ").replace("''", " ").replace("\"", " ")
    return content

def read_file_to_category_and_item_word_dict(file_name):
    lines = read_file_to_string(file_name).split("\n")
    for line in lines:
        cat = line.split(":")[0].strip()
        item = ''
        if (cat.strip() != line.replace(":", "").strip()):
            item = line.split(":")[1].strip()
        if not cat in skip_words:
            if cat in category_word_dict:
                category_word_dict[cat] += 1
            else:
                category_word_dict[cat] = 1
        if not item in skip_words:
            if item in item_word_dict:
                item_word_dict[item] += 1
            else:
                item_word_dict[item] = 1
            

category_word_dict = {}
item_word_dict = {}
for i in range(10000):
    read_file_to_category_and_item_word_dict(path_prefix+"tags_train/"+str(i)+".txt")
    

print(len(category_word_dict))
print(category_word_dict)
print(len(item_word_dict))
print(item_word_dict)

{'vehicle': 3295, 'outdoor': 1156, 'sports': 2369, 'person': 5323, 'accessory': 1880, 'furniture': 3295, 'kitchen': 3156, 'food': 1641, 'electronic': 1662, 'indoor': 1549, 'animal': 2226, 'appliance': 979}
{'airplane': 222, 'bench': 476, 'skateboard': 257, 'person': 5323, 'truck': 499, 'backpack': 429, 'handbag': 598, 'dining table': 960, 'bowl': 593, 'carrot': 139, 'spoon': 293, 'car': 1022, 'traffic light': 328, 'frisbee': 195, 'baseball bat': 194, 'bed': 315, 'chair': 1030, 'mouse': 159, 'keyboard': 170, 'book': 447, 'cup': 747, 'tv': 352, 'laptop': 304, 'donut': 118, 'bicycle': 285, 'tie': 306, 'bus': 331, 'suitcase': 189, 'baseball glove': 199, 'stop sign': 142, 'wine glass': 215, 'bottle': 708, 'clock': 442, 'snowboard': 129, 'cake': 217, 'train': 331, 'potted plant': 357, 'cat': 355, 'giraffe': 218, 'motorcycle': 328, 'bird': 306, 'dog': 392, 'surfboard': 322, 'sports ball': 340, 'fire hydrant': 152, 'teddy bear': 184, 'cell phone': 437, 'tennis racket': 294, 'sandwich': 208, 'r

In [3]:
import functools
def cmp(a, b):
    a_num = abs(float(a.split("___")[0]))
    b_num = abs(float(b.split("___")[0]))
    if (a_num < b_num):
        return -1
    return 1

In [4]:
# give each word an index so we can put them in a feature array

#load g_word_list
g_word_list = []
word_to_index = {}
tmp_list = []
for word in item_word_dict:
    tmp_list.append(str(item_word_dict[word])+"___"+word)
for word in category_word_dict:
    tmp_list.append(str(category_word_dict[word])+"___"+word)
    
tmp_list = sorted(tmp_list, key=functools.cmp_to_key(cmp))

for i in range(len(tmp_list)):
    word = tmp_list[i].split("___")[1]
    g_word_list.append(word)
    word_to_index[word] = i

# where to make the change
def my_weight_func(index, count):
    count = min(3, count)
    return 1 + (len(g_word_list)-index)*0.5/len(g_word_list) + count/3.0*0.2

def weight_occurance_array(occurance_array):
    ret = [0] * len(occurance_array)
    for index in range(len(occurance_array)):
        ret[index] = my_weight_func(index, occurance_array[index])
    return occurance_array

def read_des_to_list(path, file_num):
    ret = []
    for i in range(file_num):
        int_array = [0] * len(g_word_list)
        temp_list = read_file_to_string(path+str(i)+".txt").split()
        for word in temp_list:
            if not word in word_to_index:
                continue
            int_array[word_to_index[word]] += 1
        ret.append(weight_occurance_array(int_array))
    return ret

train_des = read_des_to_list(path_prefix+"descriptions_train/", 10000)
test_des = read_des_to_list(path_prefix+"descriptions_test/", 2000)

In [5]:
#load feature list
def load_feature_into_list(filename):
    f = open(filename, "r")
    content = f.readlines()
    f.close()
    train = filename.find("train")
    for line in content:
        name_index = int(line.split(".jpg,")[0].split("/")[1])
        feature_list = line.split(".jpg,")[1].split(",")
        if train != -1:
            train_feature_list[name_index] = feature_list
        else:
            test_feature_list[name_index] = feature_list

train_feature_list = [0] * 10000
test_feature_list = [0] * 2000
load_feature_into_list(path_prefix+"features_train/features_resnet1000_train.csv")
load_feature_into_list(path_prefix+"features_test/features_resnet1000_test.csv")

train_feature_list = np.asarray(train_feature_list, dtype=np.float64)
test_feature_list = np.asarray(test_feature_list, dtype=np.float64)

In [6]:
from sklearn.neural_network import MLPRegressor

clf = MLPRegressor(hidden_layer_sizes=(1000,1000,1000), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
clf.fit(train_des, train_feature_list)

Iteration 1, loss = 1.97395651
Iteration 2, loss = 1.49158741
Iteration 3, loss = 1.39859399
Iteration 4, loss = 1.36801217
Iteration 5, loss = 1.35083118
Iteration 6, loss = 1.33736861
Iteration 7, loss = 1.33144453
Iteration 8, loss = 1.32520659
Iteration 9, loss = 1.32147759
Iteration 10, loss = 1.31547978
Iteration 11, loss = 1.31112213
Iteration 12, loss = 1.30773539
Iteration 13, loss = 1.30351225
Iteration 14, loss = 1.30073556
Iteration 15, loss = 1.29725845
Iteration 16, loss = 1.29314432
Iteration 17, loss = 1.29030152
Iteration 18, loss = 1.29036781
Iteration 19, loss = 1.28681421
Iteration 20, loss = 1.28201404
Iteration 21, loss = 1.28041235
Iteration 22, loss = 1.27703807
Iteration 23, loss = 1.27430352
Iteration 24, loss = 1.27346802
Iteration 25, loss = 1.27115295
Iteration 26, loss = 1.26791468
Iteration 27, loss = 1.26429661
Iteration 28, loss = 1.26304574
Iteration 29, loss = 1.26089501
Iteration 30, loss = 1.25867556
Iteration 31, loss = 1.25611945
Iteration 32, los

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000, 1000, 1000), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=21, shuffle=True,
       solver='sgd', tol=1e-09, validation_fraction=0.1, verbose=10,
       warm_start=False)

In [7]:
# return a score
def compare_feature(target, given):
    return np.linalg.norm(np.array(target, dtype=float) - np.array(given, dtype=float))

def get_top_20(target):
    ret = []
    for name in range(2000):
        given = test_feature_list[name]
        score = compare_feature(target, given)
        item = str(score) + "___" + str(name)
        ret.append(item)
    return sorted(ret, key=functools.cmp_to_key(cmp))[:20]

target_feature_list = clf.predict(test_des)

f = open("submit.csv", "w")
f.write("Descritpion_ID,Top_20_Image_IDs\n")
for index in range(2000):
    target_feature = target_feature_list[index]
    twenty_image_list = get_top_20(target_feature)
    string_to_write = "" + str(index) + ".txt,"
    for name in twenty_image_list:
        file_name = name.split("___")[1] + ".jpg"+" "
        string_to_write += file_name
    string_to_write = string_to_write.strip() + "\n"
    print(string_to_write)
    f.write(string_to_write)

f.close()



0.txt,1088.jpg 68.jpg 305.jpg 1500.jpg 785.jpg 1882.jpg 1031.jpg 1231.jpg 849.jpg 1919.jpg 46.jpg 249.jpg 629.jpg 167.jpg 1018.jpg 1903.jpg 1553.jpg 1119.jpg 153.jpg 1618.jpg

1.txt,818.jpg 201.jpg 317.jpg 211.jpg 1709.jpg 1740.jpg 470.jpg 158.jpg 1017.jpg 1114.jpg 392.jpg 527.jpg 1109.jpg 1810.jpg 631.jpg 1138.jpg 1116.jpg 589.jpg 1468.jpg 1675.jpg

2.txt,1906.jpg 1998.jpg 97.jpg 993.jpg 625.jpg 1493.jpg 303.jpg 1909.jpg 980.jpg 675.jpg 990.jpg 272.jpg 437.jpg 660.jpg 1651.jpg 790.jpg 928.jpg 393.jpg 1168.jpg 406.jpg

3.txt,1088.jpg 68.jpg 305.jpg 1500.jpg 785.jpg 1882.jpg 1031.jpg 1231.jpg 849.jpg 1919.jpg 46.jpg 249.jpg 629.jpg 167.jpg 1018.jpg 1903.jpg 1553.jpg 1119.jpg 153.jpg 1618.jpg

4.txt,1088.jpg 68.jpg 305.jpg 1500.jpg 785.jpg 1882.jpg 1031.jpg 1231.jpg 849.jpg 1919.jpg 46.jpg 249.jpg 629.jpg 167.jpg 1018.jpg 1903.jpg 1553.jpg 1119.jpg 153.jpg 1618.jpg

5.txt,1913.jpg 1088.jpg 846.jpg 1254.jpg 829.jpg 673.jpg 1594.jpg 52.jpg 1700.jpg 760.jpg 1882.jpg 1693.jpg 1296.jpg 1218.j

48.txt,1088.jpg 68.jpg 305.jpg 1500.jpg 785.jpg 1882.jpg 1031.jpg 1231.jpg 849.jpg 1919.jpg 46.jpg 249.jpg 629.jpg 167.jpg 1018.jpg 1903.jpg 1553.jpg 1119.jpg 153.jpg 1618.jpg

49.txt,1475.jpg 965.jpg 663.jpg 1487.jpg 1033.jpg 1460.jpg 1211.jpg 1853.jpg 1962.jpg 1040.jpg 705.jpg 117.jpg 1031.jpg 465.jpg 607.jpg 3.jpg 1594.jpg 765.jpg 879.jpg 1119.jpg

50.txt,1362.jpg 1302.jpg 1711.jpg 183.jpg 932.jpg 1703.jpg 10.jpg 1832.jpg 1290.jpg 1123.jpg 1363.jpg 1022.jpg 560.jpg 1469.jpg 1210.jpg 987.jpg 689.jpg 1078.jpg 415.jpg 1534.jpg

51.txt,731.jpg 1207.jpg 1215.jpg 1661.jpg 1889.jpg 1930.jpg 1727.jpg 471.jpg 267.jpg 1088.jpg 1802.jpg 393.jpg 1768.jpg 1318.jpg 1430.jpg 52.jpg 249.jpg 1097.jpg 1998.jpg 155.jpg

52.txt,1137.jpg 1504.jpg 307.jpg 273.jpg 1827.jpg 1798.jpg 32.jpg 416.jpg 1342.jpg 1251.jpg 1106.jpg 285.jpg 1746.jpg 389.jpg 954.jpg 1491.jpg 1.jpg 138.jpg 1599.jpg 1112.jpg

53.txt,1906.jpg 1998.jpg 97.jpg 625.jpg 993.jpg 1493.jpg 303.jpg 1909.jpg 675.jpg 990.jpg 660.jpg 980.jpg 1168

96.txt,696.jpg 511.jpg 505.jpg 765.jpg 979.jpg 476.jpg 883.jpg 1183.jpg 1896.jpg 496.jpg 1413.jpg 338.jpg 781.jpg 1482.jpg 1611.jpg 669.jpg 1471.jpg 42.jpg 539.jpg 904.jpg

97.txt,1853.jpg 1460.jpg 1605.jpg 938.jpg 1508.jpg 399.jpg 680.jpg 1211.jpg 748.jpg 319.jpg 1739.jpg 962.jpg 398.jpg 1487.jpg 607.jpg 1483.jpg 180.jpg 1033.jpg 646.jpg 1164.jpg

98.txt,1088.jpg 68.jpg 305.jpg 1500.jpg 785.jpg 1882.jpg 1031.jpg 1231.jpg 849.jpg 1919.jpg 46.jpg 249.jpg 629.jpg 167.jpg 1018.jpg 1903.jpg 1553.jpg 1119.jpg 153.jpg 1618.jpg

99.txt,1640.jpg 1744.jpg 1150.jpg 1685.jpg 1038.jpg 686.jpg 1491.jpg 369.jpg 1157.jpg 624.jpg 1579.jpg 1319.jpg 320.jpg 1763.jpg 416.jpg 239.jpg 1251.jpg 494.jpg 976.jpg 68.jpg

100.txt,1088.jpg 68.jpg 305.jpg 1500.jpg 785.jpg 1882.jpg 1031.jpg 1231.jpg 849.jpg 1919.jpg 46.jpg 249.jpg 629.jpg 167.jpg 1018.jpg 1903.jpg 1553.jpg 1119.jpg 153.jpg 1618.jpg

101.txt,1869.jpg 383.jpg 1563.jpg 615.jpg 1359.jpg 945.jpg 1811.jpg 791.jpg 131.jpg 1800.jpg 1246.jpg 694.jpg 1498.j

143.txt,1088.jpg 68.jpg 305.jpg 1500.jpg 785.jpg 1882.jpg 1031.jpg 1231.jpg 849.jpg 1919.jpg 46.jpg 249.jpg 629.jpg 167.jpg 1018.jpg 1903.jpg 1553.jpg 1119.jpg 153.jpg 1618.jpg

144.txt,68.jpg 153.jpg 912.jpg 46.jpg 305.jpg 1243.jpg 808.jpg 216.jpg 1106.jpg 308.jpg 849.jpg 1018.jpg 1850.jpg 307.jpg 1498.jpg 1940.jpg 673.jpg 389.jpg 569.jpg 1856.jpg

145.txt,383.jpg 1869.jpg 1563.jpg 131.jpg 1498.jpg 1868.jpg 1474.jpg 945.jpg 216.jpg 1811.jpg 401.jpg 1359.jpg 395.jpg 352.jpg 791.jpg 856.jpg 956.jpg 659.jpg 1588.jpg 1478.jpg

146.txt,1088.jpg 68.jpg 305.jpg 1500.jpg 785.jpg 1882.jpg 1031.jpg 1231.jpg 849.jpg 1919.jpg 46.jpg 249.jpg 629.jpg 167.jpg 1018.jpg 1903.jpg 1553.jpg 1119.jpg 153.jpg 1618.jpg

147.txt,1088.jpg 68.jpg 305.jpg 1500.jpg 785.jpg 1882.jpg 1031.jpg 1231.jpg 849.jpg 1919.jpg 46.jpg 249.jpg 629.jpg 167.jpg 1018.jpg 1903.jpg 1553.jpg 1119.jpg 153.jpg 1618.jpg

148.txt,1375.jpg 959.jpg 112.jpg 40.jpg 108.jpg 1822.jpg 1452.jpg 1793.jpg 342.jpg 741.jpg 1843.jpg 232.jpg 1015.j

190.txt,1640.jpg 1744.jpg 1150.jpg 1685.jpg 1038.jpg 686.jpg 1491.jpg 369.jpg 1157.jpg 624.jpg 1579.jpg 1319.jpg 320.jpg 1763.jpg 416.jpg 239.jpg 1251.jpg 494.jpg 976.jpg 68.jpg

191.txt,1088.jpg 68.jpg 305.jpg 1500.jpg 785.jpg 1882.jpg 1031.jpg 1231.jpg 849.jpg 1919.jpg 46.jpg 249.jpg 629.jpg 167.jpg 1018.jpg 1903.jpg 1553.jpg 1119.jpg 153.jpg 1618.jpg

192.txt,1199.jpg 1184.jpg 80.jpg 1068.jpg 331.jpg 587.jpg 818.jpg 1408.jpg 1714.jpg 1102.jpg 850.jpg 107.jpg 1615.jpg 225.jpg 482.jpg 1143.jpg 160.jpg 428.jpg 1989.jpg 589.jpg

193.txt,725.jpg 305.jpg 1965.jpg 868.jpg 647.jpg 698.jpg 975.jpg 167.jpg 1123.jpg 549.jpg 1980.jpg 717.jpg 749.jpg 1363.jpg 10.jpg 94.jpg 393.jpg 1088.jpg 678.jpg 932.jpg

194.txt,52.jpg 620.jpg 734.jpg 914.jpg 1433.jpg 1926.jpg 1062.jpg 673.jpg 346.jpg 155.jpg 172.jpg 518.jpg 951.jpg 584.jpg 358.jpg 534.jpg 1400.jpg 670.jpg 267.jpg 1434.jpg

195.txt,1388.jpg 216.jpg 936.jpg 1243.jpg 995.jpg 1166.jpg 1451.jpg 199.jpg 978.jpg 434.jpg 1028.jpg 1462.jpg 1945.jpg 91

...

1793.txt,1088.jpg 68.jpg 305.jpg 1500.jpg 785.jpg 1882.jpg 1031.jpg 1231.jpg 849.jpg 1919.jpg 46.jpg 249.jpg 629.jpg 167.jpg 1018.jpg 1903.jpg 1553.jpg 1119.jpg 153.jpg 1618.jpg

1794.txt,1906.jpg 993.jpg 97.jpg 1493.jpg 1998.jpg 625.jpg 1909.jpg 303.jpg 980.jpg 990.jpg 790.jpg 675.jpg 272.jpg 437.jpg 1651.jpg 660.jpg 456.jpg 928.jpg 1752.jpg 229.jpg

1795.txt,1592.jpg 102.jpg 86.jpg 1873.jpg 858.jpg 40.jpg 1245.jpg 209.jpg 665.jpg 524.jpg 1912.jpg 1256.jpg 1781.jpg 1113.jpg 1914.jpg 1478.jpg 129.jpg 387.jpg 278.jpg 34.jpg

1796.txt,1369.jpg 1184.jpg 738.jpg 198.jpg 107.jpg 1839.jpg 1999.jpg 1102.jpg 482.jpg 247.jpg 1615.jpg 80.jpg 1199.jpg 668.jpg 1141.jpg 1068.jpg 225.jpg 818.jpg 1147.jpg 1507.jpg

1797.txt,809.jpg 1016.jpg 1777.jpg 1309.jpg 84.jpg 204.jpg 1375.jpg 816.jpg 108.jpg 762.jpg 621.jpg 232.jpg 176.jpg 1799.jpg 1055.jpg 139.jpg 1332.jpg 1170.jpg 911.jpg 1303.jpg

1798.txt,1018.jpg 1088.jpg 673.jpg 68.jpg 1882.jpg 87.jpg 1913.jpg 46.jpg 52.jpg 305.jpg 358.jpg 849.jpg 1154.jp

1840.txt,1317.jpg 442.jpg 123.jpg 401.jpg 1592.jpg 610.jpg 858.jpg 665.jpg 1770.jpg 1897.jpg 352.jpg 1546.jpg 1519.jpg 193.jpg 1399.jpg 606.jpg 40.jpg 1963.jpg 503.jpg 524.jpg

1841.txt,1914.jpg 1113.jpg 232.jpg 102.jpg 793.jpg 935.jpg 1684.jpg 1912.jpg 1090.jpg 108.jpg 1658.jpg 142.jpg 1780.jpg 209.jpg 34.jpg 1478.jpg 1332.jpg 524.jpg 387.jpg 1245.jpg

1842.txt,1088.jpg 68.jpg 305.jpg 1500.jpg 785.jpg 1882.jpg 1031.jpg 1231.jpg 849.jpg 1919.jpg 46.jpg 249.jpg 629.jpg 167.jpg 1018.jpg 1903.jpg 1553.jpg 1119.jpg 153.jpg 1618.jpg

1843.txt,198.jpg 738.jpg 1369.jpg 1839.jpg 1102.jpg 1184.jpg 80.jpg 1615.jpg 107.jpg 247.jpg 1199.jpg 482.jpg 1507.jpg 1999.jpg 1147.jpg 818.jpg 149.jpg 668.jpg 6.jpg 1068.jpg

1844.txt,1088.jpg 68.jpg 305.jpg 1500.jpg 785.jpg 1882.jpg 1031.jpg 1231.jpg 849.jpg 1919.jpg 46.jpg 249.jpg 629.jpg 167.jpg 1018.jpg 1903.jpg 1553.jpg 1119.jpg 153.jpg 1618.jpg

1845.txt,68.jpg 1527.jpg 1257.jpg 1076.jpg 1977.jpg 674.jpg 164.jpg 912.jpg 211.jpg 1243.jpg 1816.jpg 916.jpg

1887.txt,433.jpg 1709.jpg 640.jpg 1431.jpg 1109.jpg 701.jpg 1704.jpg 1065.jpg 317.jpg 1138.jpg 500.jpg 1326.jpg 585.jpg 1527.jpg 1675.jpg 201.jpg 1988.jpg 171.jpg 1093.jpg 1740.jpg

1888.txt,590.jpg 1138.jpg 1891.jpg 201.jpg 1988.jpg 611.jpg 326.jpg 392.jpg 875.jpg 571.jpg 572.jpg 635.jpg 774.jpg 888.jpg 238.jpg 1353.jpg 1347.jpg 355.jpg 179.jpg 1754.jpg

1889.txt,1885.jpg 1768.jpg 1815.jpg 1613.jpg 1377.jpg 1790.jpg 1216.jpg 402.jpg 582.jpg 304.jpg 367.jpg 16.jpg 1607.jpg 1139.jpg 1430.jpg 1701.jpg 1122.jpg 258.jpg 718.jpg 1151.jpg

1890.txt,816.jpg 736.jpg 484.jpg 574.jpg 435.jpg 395.jpg 1639.jpg 595.jpg 946.jpg 942.jpg 1924.jpg 1076.jpg 533.jpg 425.jpg 961.jpg 1822.jpg 1131.jpg 1257.jpg 299.jpg 635.jpg

1891.txt,1913.jpg 1700.jpg 846.jpg 1594.jpg 1296.jpg 760.jpg 829.jpg 476.jpg 82.jpg 1088.jpg 998.jpg 1553.jpg 1118.jpg 249.jpg 338.jpg 1191.jpg 1693.jpg 1980.jpg 1882.jpg 1254.jpg

1892.txt,1356.jpg 1545.jpg 1007.jpg 39.jpg 1873.jpg 1490.jpg 1478.jpg 924.jpg 733.jpg 780.jpg 1554.jpg 

1933.txt,548.jpg 9.jpg 722.jpg 1221.jpg 1975.jpg 988.jpg 1257.jpg 4.jpg 1951.jpg 1182.jpg 966.jpg 1299.jpg 1271.jpg 470.jpg 164.jpg 1359.jpg 383.jpg 919.jpg 713.jpg 746.jpg

1934.txt,1076.jpg 1726.jpg 726.jpg 12.jpg 7.jpg 877.jpg 118.jpg 1639.jpg 562.jpg 1679.jpg 1110.jpg 1822.jpg 1753.jpg 198.jpg 152.jpg 595.jpg 76.jpg 355.jpg 587.jpg 818.jpg

1935.txt,635.jpg 1474.jpg 1784.jpg 613.jpg 839.jpg 1631.jpg 856.jpg 1099.jpg 1908.jpg 150.jpg 211.jpg 1478.jpg 956.jpg 733.jpg 924.jpg 1142.jpg 1735.jpg 1619.jpg 1957.jpg 1935.jpg

1936.txt,1088.jpg 68.jpg 305.jpg 1500.jpg 785.jpg 1882.jpg 1031.jpg 1231.jpg 849.jpg 1919.jpg 46.jpg 249.jpg 629.jpg 167.jpg 1018.jpg 1903.jpg 1553.jpg 1119.jpg 153.jpg 1618.jpg

1937.txt,1180.jpg 1902.jpg 1262.jpg 3.jpg 597.jpg 459.jpg 563.jpg 985.jpg 1011.jpg 1663.jpg 36.jpg 199.jpg 604.jpg 46.jpg 669.jpg 1581.jpg 1446.jpg 360.jpg 231.jpg 1163.jpg

1938.txt,448.jpg 1249.jpg 1488.jpg 1857.jpg 1285.jpg 689.jpg 987.jpg 1961.jpg 1694.jpg 1085.jpg 1270.jpg 685.jpg 975.jp

1981.txt,1356.jpg 780.jpg 733.jpg 1478.jpg 1873.jpg 1422.jpg 1554.jpg 324.jpg 491.jpg 1964.jpg 129.jpg 1007.jpg 1349.jpg 1545.jpg 1490.jpg 827.jpg 278.jpg 578.jpg 1058.jpg 291.jpg

1982.txt,1088.jpg 68.jpg 305.jpg 1500.jpg 785.jpg 1882.jpg 1031.jpg 1231.jpg 849.jpg 1919.jpg 46.jpg 249.jpg 629.jpg 167.jpg 1018.jpg 1903.jpg 1553.jpg 1119.jpg 153.jpg 1618.jpg

1983.txt,1853.jpg 1460.jpg 748.jpg 1605.jpg 1739.jpg 1211.jpg 607.jpg 938.jpg 646.jpg 398.jpg 1487.jpg 680.jpg 1508.jpg 1164.jpg 319.jpg 962.jpg 399.jpg 1033.jpg 1027.jpg 1962.jpg

1984.txt,198.jpg 191.jpg 1636.jpg 1184.jpg 587.jpg 1839.jpg 1369.jpg 1192.jpg 674.jpg 1147.jpg 1102.jpg 76.jpg 1172.jpg 635.jpg 1375.jpg 1158.jpg 1615.jpg 504.jpg 668.jpg 738.jpg

1985.txt,1919.jpg 981.jpg 633.jpg 1960.jpg 1330.jpg 1547.jpg 1553.jpg 1057.jpg 110.jpg 192.jpg 1720.jpg 55.jpg 739.jpg 371.jpg 153.jpg 1656.jpg 1500.jpg 749.jpg 1024.jpg 290.jpg

1986.txt,961.jpg 1271.jpg 462.jpg 596.jpg 1359.jpg 1332.jpg 1531.jpg 216.jpg 83.jpg 1293.jpg 876.jpg